Se procede a combinar los datos de CO2 con las zonas usando shape-leng y shape-area, ya que,en el archivo taxi_zones.dbf están las columnas:
Shape-Leng: Es la longitud del perímetro del polígono (zona).
Shape-Area: Es el área del polígono recorrido.Y en el archivo Vehicle Fuel Economy Data.csv se encuentran las columnas de:
combustible utilizado en cada auto.Por lo tanto se hara una conversion que me permita sacar las emisiones de CO2 y asociarlas a los km recorridos.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno 
import warnings
from dbfread import DBF


Ignorar mensajes de advertencias

In [3]:
warnings.filterwarnings("ignore")

Ingesta y lectura de los archivos.

In [12]:
# Leer el archivo .dbf
dbf_path = r'C:\\Users\\walter\\Desktop\\Proyecto-NYC-Taxis\\Data\\taxi_zones.dbf'
table = DBF(dbf_path)




In [13]:
# Convertir a un DataFrame de pandas
df_zonas = pd.DataFrame(iter(table))

In [14]:
print(df_zonas.head())

   OBJECTID  Shape_Leng  Shape_Area                     zone  LocationID  \
0         1    0.116357    0.000782           Newark Airport           1   
1         2    0.433470    0.004866              Jamaica Bay           2   
2         3    0.084341    0.000314  Allerton/Pelham Gardens           3   
3         4    0.043567    0.000112            Alphabet City           4   
4         5    0.092146    0.000498            Arden Heights           5   

         borough  
0            EWR  
1         Queens  
2          Bronx  
3      Manhattan  
4  Staten Island  


In [29]:
df_co2 = pd.read_csv(r'C:\Users\walter\Desktop\Proyecto-NYC-Taxis\Data\Vehicle Fuel Economy Data.csv')
print(df_co2.head())


   Year Manufacturer                          Model  barrels08  barrelsA08  \
0  2017          BMW                       M6 Coupe  17.500588         0.0   
1  2017          BMW                   X3 sDrive28i  12.396250         0.0   
2  2016     Infiniti                 Q50S Red Sport  13.523182         0.0   
3  2017         MINI  John Cooper Works Convertible  11.018889         0.0   
4  2017          Kia                      Sedona SX  14.167143         0.0   

   charge240  city08  city08U  cityA08  cityA08U  ...  mfrCode  c240Dscr  \
0        0.0    15.0     14.0      0.0       0.0  ...      BMX       NaN   
1        0.0    21.0     21.0      0.0       0.0  ...      BMX       NaN   
2        0.0    20.0     19.0      0.0       0.0  ...      NSX       NaN   
3        0.0    24.0     23.0      0.0       0.0  ...      BMX       NaN   
4        0.0    18.0     18.0      0.0       0.0  ...      KMX       NaN   

   charge240b  c240bDscr   createdOn  modifiedOn  startStop  phevCity  \
0

Armar el consumo generado

In [40]:
# DataFrame con rendimiento en ciudad
data = {
    'Year': [2017, 2017, 2016],
    'Manufacturer': ['BMW', 'BMW', 'Infiniti'],
    'Model': ['M6 Coupe', 'X3 sDrive28i', 'Q50S Red Sport'],
     'barrels08': [17.5, 12.4, 13.5],
     'barrelsA08': [0.0, 0.0, 0.0]  
}



In [41]:
df_combustible = pd.DataFrame(data)

In [44]:
# Calcula el total de combustible
df_combustible['Total_Fuel'] = df_combustible['barrels08'] + df_combustible['barrelsA08']



In [45]:
model_to_zone = {
    'M6 Coupe': 'Zone',
    'X3 sDrive28i': 'Zone',
    'Q50S Red Sport': 'Zone'
}


In [46]:
# Se asocia el modelo con la zona
df_combustible['Zone'] = df_combustible['Model'].map(model_to_zone)


In [47]:
# Se agrupa por zona y calcula el total de combustible
df_zone_fuel = df_combustible.groupby('Zone')['Total_Fuel'].sum().reset_index()

In [48]:
print(df_zone_fuel)

   Zone  Total_Fuel
0  Zone        43.4


In [49]:
# Total de combustible en galones
total_combustible_galones = 43.4

# Tasa de emisión de CO₂ en toneladas por galón
tasa_emision = 0.00889

# Calcular las emisiones totales
emisiones_totales = total_combustible_galones * tasa_emision
print(f'Emisiones Totales en Toneladas: {emisiones_totales}')


Emisiones Totales en Toneladas: 0.385826


In [68]:

# Datos de zonas con emisiones calculadas
df_zone_fuel = pd.DataFrame({
    'Zone': ['Newark Airport', 'Jamaica Bay', 'Allerton/Pelham Gardens', 'Alphabet City', 'Arden Heights'],
    'Total_Fuel': [43.4 / 5] * 5,  # Combustible distribuido uniformemente
    'CO2_Emissions': [43.4 / 5 * 0.00889] * 5  # Emisiones calculadas
})

# Datos de distritos
df_zonas_distritos = pd.DataFrame({
    'Zone': ['Newark Airport', 'Jamaica Bay', 'Allerton/Pelham Gardens', 'Alphabet City', 'Arden Heights'],
    'District': ['EWR', 'Queens', 'Bronx', 'Manhattan', 'Staten Island']
})

# Une los datos de zonas con los distritos
df_zonas_distritos = pd.merge(df_zonas_distritos, df_zone_fuel[['Zone', 'CO2_Emissions']], on='Zone', how='left')

# Calcula la proporción de emisiones
df_zonas_distritos['Proporcion_Emisiones'] = df_zonas_distritos['CO2_Emissions'] / df_zonas_distritos['CO2_Emissions'].sum()

# Distribuye las emisiones entre los distritos
df_zonas_distritos['District_CO2_Emissions'] = df_zonas_distritos['Proporcion_Emisiones'] * df_zonas_distritos['CO2_Emissions']

print("\nDistribución de emisiones entre distritos:")
print(df_zonas_distritos)


Distribución de emisiones entre distritos:
                      Zone       District  CO2_Emissions  \
0           Newark Airport            EWR       0.077165   
1              Jamaica Bay         Queens       0.077165   
2  Allerton/Pelham Gardens          Bronx       0.077165   
3            Alphabet City      Manhattan       0.077165   
4            Arden Heights  Staten Island       0.077165   

   Proporcion_Emisiones  District_CO2_Emissions  
0                   0.2                0.015433  
1                   0.2                0.015433  
2                   0.2                0.015433  
3                   0.2                0.015433  
4                   0.2                0.015433  


Se redondea a 3 decimales 

In [69]:
# Redondeo
df_zonas_distritos = df_zonas_distritos.round(3)

print("\nDistribución de emisiones entre distritos con valores redondeados:")
print(df_zonas_distritos)


Distribución de emisiones entre distritos con valores redondeados:
                      Zone       District  CO2_Emissions  \
0           Newark Airport            EWR          0.077   
1              Jamaica Bay         Queens          0.077   
2  Allerton/Pelham Gardens          Bronx          0.077   
3            Alphabet City      Manhattan          0.077   
4            Arden Heights  Staten Island          0.077   

   Proporcion_Emisiones  District_CO2_Emissions  
0                   0.2                   0.015  
1                   0.2                   0.015  
2                   0.2                   0.015  
3                   0.2                   0.015  
4                   0.2                   0.015  


Se guarda en archivo csv manejable

In [70]:
# Guardar el DataFrame en un archivo CSV
df_zonas_distritos.to_csv('zonas_distritos_emisiones.csv', index=False)


Se rellee para confirmación que no se hayan modificado ningun dato

In [71]:
df_zonas_distritos_leido = pd.read_csv('zonas_distritos_emisiones.csv')

print(df_zonas_distritos_leido)


                      Zone       District  CO2_Emissions  \
0           Newark Airport            EWR          0.077   
1              Jamaica Bay         Queens          0.077   
2  Allerton/Pelham Gardens          Bronx          0.077   
3            Alphabet City      Manhattan          0.077   
4            Arden Heights  Staten Island          0.077   

   Proporcion_Emisiones  District_CO2_Emissions  
0                   0.2                   0.015  
1                   0.2                   0.015  
2                   0.2                   0.015  
3                   0.2                   0.015  
4                   0.2                   0.015  


Conclusión :
Los valores redondeados a 3 decimales proporcionan una representación precisa y manejable de las emisiones en cada distrito, lo que facilita su análisis y visualización en herramientas como Power BI.